In [65]:
def function1(input1:str):
    return input1 + " Hi " 

def function2(input2:str)->str:
    return input2 + "There"

In [66]:
from langgraph.graph import Graph

workflow=Graph()

workflow.add_node("node1" , function1)
workflow.add_node("node2" , function2)

workflow.add_edge("node1" , "node2")

workflow.set_entry_point("node1")
workflow.set_finish_point("node2")

app=workflow.compile()

In [67]:
app.invoke("Hello")

'Hello Hi There'

In [68]:
input="hello"

for output in app.stream(input):
    for key , value in output.items():
        print(f"The output from node {key} : ")
        print("------------")
        print(f"{value}")

The output from node node1 : 
------------
hello Hi 
The output from node node2 : 
------------
hello Hi There


In [69]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
os.getenv("GROQ_API_KEY")
os.getenv("TAVILY_API_KEY")
groq_llm=ChatGroq(model="Mixtral-8x7b-32768")

In [70]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()


In [71]:
Agent_State={}

Agent_State["messages"]=[]
print(Agent_State)

{'messages': []}


In [78]:
def function3(state)->str:
    messages=state['messages']
    input3=messages[-1]
    role="Your task is to take the information out of the car which will include the brand name and the car model , NOTE!! NOTHING ELSE THEN CAR MODEL AND CAR BRAND . HERE IS THE USER QUERY : \n" + input3
    response3=groq_llm.invoke(role)
    state['messages'].append(response3)
    return response3.content

def function4(state)->str:
    custom_contents=[]
    messages=state["messages"]
    input4=messages[-1]
    response4=search.run(input4)

    for result in response4:
        content=result.get('content' , '')
        if content:
            custom_contents.append(content)
    if custom_contents:
        state['messages'].append(custom_contents)
        return "Search Results :" + ' '.join(custom_contents)
    
    else:
        state['messages'].append("Search Results : No content found")
        return "Search Results : No content found" 

def function5(state):
    messages=state['messages']
    user_input=messages[0]
    available_info=messages[1]
    agent_query2="Your task is to match if the what the user has demanded in his query which is this \n"+ user_input + " \n with the information retrieved which is: \n" + available_info +"so basically your task is to see if the information is correct or not"
    response5=groq_llm.invoke(agent_query2)
    return response5.content
workflow2= Graph()

workflow2.add_node("agent" , function3)
workflow2.add_node("tool" , function4)
workflow2.add_node("evaluation" , function5)

workflow2.add_edge("agent" , "tool")
workflow2.add_edge("tool" , "evaluation")

workflow2.set_entry_point("agent")
workflow2.set_finish_point("evaluation")

app2=workflow2.compile()

inputs={"messages": ["What do you know about mercedes c63s 6.2l v8 , when was it released and what is its price?"]}
app2.invoke(inputs)


TypeError: string indices must be integers, not 'str'

In [ ]:
input="What do you know about mercedes c63s"

for output in app2.stream(input):
    for key , value in output.items():
        print(f"The output from node {key} is \n ")
        print("----------------------- \n ")
        print(f"{value}")

The output from node node3 is 
 
----------------------- 
 
The car brand is Mercedes and the car model is C63S.
The output from node node4 is 
 
----------------------- 
 
AGENT SAYS :Specifications
2024 Mercedes-AMG C63 S E-Performance
Vehicle Type: front-engine, rear-motor, all-wheel-drive, 5-passenger, 4-door sedan
PRICE (C/D EST)
Base: $95,000
POWERTRAINSTurbocharged and intercooled DOHC 16-valve inline-4, 469 hp, 402 lb-ft + AC motor, 201 hp, 236 lb-ft (combined output: 671 hp, 752 lb-ft; 4.8-kWh lithium-ion battery pack; 3.7-kW onboard charger)
Transmissions F/R: 9-speed automatic, 2-speed automatic
DIMENSIONS
Wheelbase: 113.2 in
Length: 190.6 in
Width: 74.8 in
Height: 57.4 in
Passenger Volume: 94 ft3
Trunk Volume: 10 ft3
Curb Weight (C/D est): 4700 lb
PERFORMANCE (C/D EST)
60 mph: 3.0 sec
100 mph: 6.7 sec
1/4-Mile: 10.8 sec
Top Speed: 174 mph
EPA FUEL ECONOMY (C/D EST)
Combined/City/Highway: 21/18/25 mpg
Combined Gasoline + Electricity: 35 MPGe
EV Range: 6 mi
Shopping Tools
Mer